In [1]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
import time
from IPython.display import display, clear_output
from fastai.data.all import *
import pandas as pd
from pathlib import Path
import regex as re
import numpy as np
from collections.abc import Sequence


In [2]:
#export
def show_frames(video,start=0, end=5):
    '''show frames in a video from start to end'''
    for frame in video[start:end]:
        clear_output(wait=True)
        frame.show()
        time.sleep(0.5)


In [3]:
#export
class Video(L):
    ''' the init function takes a list of PILImage s'''  
    @classmethod
    def create(cls, paths:(L,list,str), sep='\n'): 
        '''create images from frames path in a video'''
        paths = paths.split(sep) if isinstance(paths, str) else paths
        return cls(map(PILImage.create, paths))
        

In [5]:
#export
def snippets_from_video(vid, l=10, s=2):
    '''create list of snippet out a video'''
    vid=vid[::s] # skip frames
    return [[vid[i] for i in range(k*l, k*l + l)] for k in range(0,len(vid)//l)]

class ResizeTime(Transform):
    split_idx = None # 0- train 1- validation 
    def __init__(self, skip=2, l=50, drop_last=True,**kwargs): 
        self.skip = skip
        self.l = l
        self.drop_last = drop_last
        super().__init__(**kwargs)
        
    def encodes(self, vid:Video, split_idx=split_idx):
        '''create a list of frame-images (snippet) out a single video path'''
        l, skip = self.l, self.skip
        snippet_list = snippets_from_video(vid,s=skip,l=l)
        idx = len(snippet_list)//2 if split_idx else random.randint(0,len(snippet_list)-1) # ** if validation always takes middle snip
        return Video(snippet_list[idx])

In [6]:
#export
@ToTensor
def encodes(self, vid:Video):
    img2tens=ToTensor()
    return torch.cat([img2tens(frame)[None] for frame in vid])


In [7]:
#export
def _get_sz(x):
    if isinstance(x, tuple): x = x[0]
    if not isinstance(x, Tensor): return fastuple(x.size)
    return fastuple(x.get_meta('img_size', x.get_meta('sz', (x.shape[-1], x.shape[-2]))))

@Resize
def encodes(self, video:Video):
        nw_vid=[]
        for frame in video:
            orig_sz = _get_sz(frame)
            w,h = orig_sz
            op = (operator.lt,operator.gt)[self.method==ResizeMethod.Pad]
            m = w/self.size[0] if op(w/self.size[0],h/self.size[1]) else h/self.size[1]
            cp_sz = (int(m*self.size[0]),int(m*self.size[1]))
            tl = fastuple(int(self.pcts[0]*(w-cp_sz[0])), int(self.pcts[1]*(h-cp_sz[1])))
            fastaiImg = PILImage.create(np.array(frame.crop_pad(cp_sz, tl, orig_sz=orig_sz, pad_mode=self.pad_mode,
                       resize_mode=self.mode_mask if isinstance(frame,PILMask) else self.mode, resize_to=self.size)))
            nw_vid.append(fastaiImg)
        return Video(nw_vid)


In [65]:
#export
def _uniformize_dataset(items:pd.DataFrame, n_el=3, n_lbl=3, shuffle=True):
    lbls = list(items['lbl'])
    vocab = list(set(lbls))
    lbl2idxs = {lbl:[] for lbl in vocab}
    for i, lbl in enumerate(lbls): lbl2idxs[lbl].append(i)
    if shuffle: [random.shuffle(v) for v in lbl2idxs.values()]
    idxs = []
    while len(vocab) >= n_lbl:
        lbl_samples = random.sample(vocab, n_lbl)
        for lbl in lbl_samples:
            for i in range(n_el):
                idx = lbl2idxs[lbl].pop()
                idxs.append(idx)
            if len(lbl2idxs[lbl]) <= n_el:
                vocab.remove(lbl)
    return idxs

def uniformize_dataset(items:pd.DataFrame, n_el=3, n_lbl=3, shuffle=True):
    idxs = _uniformize_dataset(items=items, n_el=n_el, n_lbl=n_lbl, shuffle=shuffle)
    return items.iloc[idxs]

In [73]:
path_charades = Path('/mnt/data/adrianlopez/Videos/Charades') 
items = uniformize_dataset(pd.read_csv(path_charades/'df0.csv', index_col=0))
items.head(6)

,id,coded_lbl,lbl,start,end,test
64212,LD6TD,c146,Someone is awakening somewhere,5.4,10.3,True
32065,CSLEP,c146,Someone is awakening somewhere,2.3,10.2,False
19186,785X2,c146,Someone is awakening somewhere,20.5,30.7,False
64667,7KW31,c099,Putting a broom somewhere,1.6,6.9,True
13559,NIRNP,c099,Putting a broom somewhere,6.8,12.7,False
62398,00607,c099,Putting a broom somewhere,25.9,31.7,True


In [10]:
#export
class UniformizedShuffle():
    def __init__(self, items, n_el=3, n_lbl=3):
        store_attr(self, 'items,n_el,n_lbl')
    
    def __call__ (self, x=None):
        return _uniformize_dataset(items=self.items, n_el=self.n_el, n_lbl=self.n_lbl)
        

In [74]:
path_charades = Path('/mnt/data/adrianlopez/Videos/Charades') 
items = pd.read_csv(path_charades/'df0.csv', index_col=0)
un = UniformizedShuffle(items)
idxs = un()
items.iloc[idxs].tail(6)

,id,coded_lbl,lbl,start,end,test
1822,U6KQ7,c154,Someone is standing up from somewhere,2.8,8.4,False
1802,MUAR4,c154,Someone is standing up from somewhere,24.0,30.4,False
1767,29T54,c154,Someone is standing up from somewhere,1.1,7.9,False
281,DWHPO,c059,Sitting in a chair,0.0,31.0,False
188,PSPMA,c059,Sitting in a chair,0.0,32.0,False
129,BDFDE,c059,Sitting in a chair,7.3,33.0,False


In [ ]:
def get_snippets(path_to_csv, col_name='paths', skip=3, l=25, sep='\n'):
        '''create a list of all snippets in path form'''
        df=pd.read_csv(path_to_csv)
        vds=[v.split(sep) for v in df[col_name]] #  list of frames in a list of all the videos
        ll_snip=[snippets_from_video(el,l,skip) for el in vds] # list of videos cont list of snips cont list of frames 
        return [s for l_snip in ll_snip for s in l_snip] #flatten list

In [76]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_datasets.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp video_block